In [18]:
import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler

In [5]:
def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.

if os.environ.get('RUNTIME_ENV_LOCATION_TYPE') == 'external':
    endpoint_22c70bf955fd4cebae213e416536e318 = 'https://s3-api.us-geo.objectstorage.softlayer.net'
else:
    endpoint_22c70bf955fd4cebae213e416536e318 = 'https://s3-api.us-geo.objectstorage.service.networklayer.com'

client_22c70bf955fd4cebae213e416536e318 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='1MyMhnkW8h5Udl5m2IySghFCfaupYijxayJEy1diCYRS',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url=endpoint_22c70bf955fd4cebae213e416536e318)

body = client_22c70bf955fd4cebae213e416536e318.get_object(Bucket='latihanwatsonstudio-donotdelete-pr-cnb4lwychwpcrp',Key='Social_Network_Ads.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df = pd.read_csv(body)
df.head()

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   User ID          400 non-null    int64 
 1   Gender           400 non-null    object
 2   Age              400 non-null    int64 
 3   EstimatedSalary  400 non-null    int64 
 4   Purchased        400 non-null    int64 
dtypes: int64(4), object(1)
memory usage: 15.8+ KB


In [8]:
# drop kolom yang tidak diperlukan
data = df.drop(columns=['User ID'])

# jalankan proses one-hot encoding dengan pd.get_dumies()
data = pd.get_dummies(data)
data

,Age,EstimatedSalary,Purchased,Gender_Female,Gender_Male
0,19,19000,0,0,1
1,35,20000,0,0,1
2,26,43000,0,1,0
3,27,57000,0,1,0
4,19,76000,0,0,1
...,...,...,...,...,...
395,46,41000,1,1,0
396,51,23000,1,0,1
397,50,20000,1,1,0
398,36,33000,0,0,1


In [11]:
# pisahkan atribut dan label
predictions = ['Age', 'EstimatedSalary', 'Gender_Female', 'Gender_Male']

x = data[predictions]
y = data['Purchased']

# lakukan normalisasi terhadap data yang kita miliki
scaler = StandardScaler()
scaler.fit(x)

scaled_data = scaler.transform(x)
scaled_data = pd.DataFrame(scaled_data, columns = x.columns)
scaled_data.head()

,Age,EstimatedSalary,Gender_Female,Gender_Male
0,-1.781797,-1.490046,-1.020204,1.020204
1,-0.253587,-1.460681,-1.020204,1.020204
2,-1.113206,-0.785290,0.980196,-0.980196
3,-1.017692,-0.374182,0.980196,-0.980196
4,-1.781797,0.183751,-1.020204,1.020204


In [23]:
# bagi data menjadi train dan test untuk setiap atribut dan label
x_train, x_test, y_train, y_test = train_test_split(scaled_data, y, test_size = 0.2, random_state = 1)


# latih model dengan fungsi fit
model = linear_model.LogisticRegression()
model.fit(x_train, y_train)

"""
saat uji akurasi model seumpama score menghasilkan 0.825 atau 82.5% 
itu artinya tebakan yang dilakukan oleh model ketika melakukan test di 0.2 dataset 
atau test size adalah 82.5% benar dan 17.5% salah
"""

# uji akurasi model
model.score(x_test, y_test)

0.825